# Wikidata csv to xml converter

In [1]:
# read in data
path_kev = '../data/wikidata/wd_species_df_kev.csv'
path_til = '../data/wikidata/wd_species_df_tillman.csv'


# import
from wd_species import *
import pandas as pd

## Dataframe Preprocessing 
Combine dataframes and deduplicate them

In [2]:
# read in and concaneate dataframes
df = pd.read_csv(path_kev)
df = pd.concat([df, pd.read_csv(path_til)])


print('Dataframe length before deduplication: ', df.shape[0])

# deduplicate dataframes
df = df.drop_duplicates()


print('Dataframe length after deduplication: ', df.shape[0])

# write to dataframe path
df.to_csv(getPathDF(),  index = False)

Dataframe length before deduplication:  152399
Dataframe length after deduplication:  150724


## XML creation

In [3]:
createXML()